In [ ]:
import sqlite3

# Abrimos la conexión con la BBDD
conn = sqlite3.connect("D:/master/Master-in-Data-Science/Proyecto/Datos/worldTimeSeries.sqlite")
cursor = conn.cursor()

# Vamos a intentar automatizar el análisis anterior, es decir, considerar los datos a partir de cierto año y a 
# continuacion filtrar por aquellas estaciones que tienen observaciones por año inferior a 364 dias
df_total = DataFrame()
#continentes=["europa"]
continentes=["antarctica","central_america","south_america","europa","africa","asia","oceania","north_america"]

for cont in continentes:
    print("Inicio continente: " + cont + " a las " + str(time.strftime("%H:%M:%S")))
    
    # Recuperamos todos los paises que aparecen en el continente
    countries=[]
    rows = cursor.execute("select distinct country from world_time_"+cont+";")
    for r in rows:
        countries.append(r[0])

    for country in countries:

        
        
    # Se recuperan todas las estaciones del continentes que existían previamente a 1900 y existen en 2016
    rows = cursor.execute("select distinct station from world_time_"+cont+" where date = '19000101';")
        
    station_list_1900 = []
    for r in rows:
        station_list_1900.append({"station":r[0]})
        
    if len(station_list_1900) > 0:
        rows = cursor.execute("select distinct station from world_time_"+cont+" where date = '20160401';")        
        station_list_2016 = []
        for r in rows:
            station_list_2016.append({"station":r[0]})

        station_list_1900 = DataFrame(station_list_1900)
        station_list_2016 = DataFrame(station_list_2016)

        # Nos quedamos con las estaciones que estan en ambas listas

        station_list = station_list_1900.merge(station_list_2016)
        station_list = station_list.reset_index()

        if len(station_list) > 0:
            # Se compone la lista de estaciones para que lo entienda SQL, se seleccionan bloques de 500 estaciones
            stations = []
            stats = ""
            n = 0;
            for s in station_list.as_matrix():
                n = n + 1
                if stats != "":
                    stats = stats + ","
                stats = stats + "'" + s[1] + "'"
                
                if n % 100 == 0:
                    stations.append(stats)
                    stats = ""
                    
            if stats != "":
                stations.append(stats)

            # Se itera para cada uno de los bloques de 100 estaciones
            n_lote = 0
            while n_lote < len(stations):
                print ("   Iteracion sobre el bloque " + str(n_lote) + " de 100 estaciones de " + str(len(stations)))
                rows = cursor.execute("""select 
                                        station,
                                        date,
                                        sum(TMAX) as TMAX, 
                                        sum(TMIN) as TMIN, 
                                        sum(PRCP) as PRCP,
                                        count(*) as NOBS
                                    from world_time_"""+cont+""" 
                                    where station in ("""+stations[n_lote]+""") 
                                    group by station, date
                                    order by station, date;""")                

                result=[]           
                for r in rows:
                    if int(r[1][:4]) >= 1900:
                        result.append({"station":r[0],"date":r[1],"TMAX":r[2],"TMIN":r[3],"PRCP":r[4],"NOBS":r[5]})

                df = DataFrame(result)
                
                result = []
                n_lote = n_lote+1
                
                df.PRCP = df.PRCP.fillna(0)
                df = df.dropna()

                # Para reducir el tiempo que consume la query anterior, se hace la media anual de temperaturas 
                # fuera del SQL
                # Se calcula el año a partir de la fecha
                df['year'] = df.date.map(lambda x: int(x[:4]))

                # Analizamos el numero de observaciones por estacion
                df['n'] = 1
                df_agg = df.groupby(["station","year"]).sum()
                df_agg = df_agg.reset_index()
                df = DataFrame()

                # Identificamos todas aquellas estaciones que tienen años con un numero de observaciones inferior 
                # a 364, excluyendo para ello el año 2016 (que no esta completo)
                df_falta_datos = df_agg[df_agg.n < 364]
                df_falta_datos = df_falta_datos[df_falta_datos.year != 2016]
                df_falta_datos = df_falta_datos.drop_duplicates("station")

                # Se eliminan las estaciones identificadas
                for s in df_falta_datos.station:
                    df_agg = df_agg[df_agg.station != s]

                # Se eliminan las variables no necesarias
                df_agg = df_agg[['station','year','TMAX','TMIN','PRCP','NOBS']]

                # Es necesario eliminar las series que tienen un año completo sin datos

                # Agregamos los indicadores por estacion
                df_agg_2 = df_agg.groupby('station').count()

                # Identificamos aquellas estaciones que no llegan al número de años que deberían tener
                if len(df_agg_2) > 0:
                    df_agg_2 = df_agg_2[df_agg_2.year < max(df_agg_2.year)]

                df_agg_2 = df_agg_2.reset_index()[['station']]

                # Eliminamos las estaciones identificadas
                for s in df_agg_2.station:
                    df_agg = df_agg[df_agg.station != s]

                df_agg_2 = DataFrame()
                # Agregamos los indicadores por año
                df_agg = df_agg.groupby('year').sum()
                df_agg = df_agg.reset_index()
                
                df_agg['continent'] = cont
                
                df_total = df_total.append(df_agg)
                df_agg = DataFrame()
                df_total.to_csv("Resultado_calentamiento.csv")
    
    print("Fin continente: " + cont)    
    
    df_total.to_csv("Resultado_calentamiento.csv")
conn.close()

# Una vez iterados por todos los continentes, se realiza una agrupacion por continente y año para obtener los 
# indicadores esperados

df_total = df_total.groupby(['continent','year']).sum()
df_total = df_total.reset_index()

# Nos quedamos con las observaciones posteriores a 1900
df_total = df_total[df_total.year >= 1900]

# Se calculan los indicadores
df_total.TMAX = df_total.TMAX/df_total.NOBS
df_total.TMIN = df_total.TMIN/df_total.NOBS
df_total.PRCP = df_total.PRCP/df_total.NOBS

# Se almacena el resultado final
df_total.to_csv("Resultado_calentamiento.csv")
